In [26]:
import pandas as pd
import numpy as np
import bokeh

One-hot encoding for categorical columns with get_dummies

In [27]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

Preprocess application_train.csv and application_test.csv

In [28]:
#def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
df = pd.read_csv('../datasets/application_train.csv')
test_df = pd.read_csv('../datasets/application_test.csv')
print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))

Train samples: 307511, test samples: 48744


In [29]:

df = df.append(test_df).reset_index()
# Optional: Remove 4 applications with XNA CODE_GENDER (train set)
df = df[df['CODE_GENDER'] != 'XNA']

# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])
# Categorical features with One-Hot encode
df, cat_cols = one_hot_encoder(df, nan_as_category=True)

# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    #del test_df
    #gc.collect()
    #return df

C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17036\3700531176.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(test_df).reset_index()


Preprocess bureau.csv and bureau_balance.csv

Preprocess previous_applications.csv

Preprocess POS_CASH_balance.csv

Preprocess credit_card_balance.csv

LightGBM GBDT with KFold or Stratified KFold<br>
Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code

In [30]:
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,0,1,0,0,0.067329,0.498036,202500.0,0.121978,0.060749
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,0,1,0,0,0.070862,0.208736,135000.0,0.132217,0.027598
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,0,1,0.011814,0.500000,67500.0,0.100000,0.050000
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,1,0,0,1,0.159905,0.431748,67500.0,0.219900,0.094941
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,0,1,0.152418,0.236842,121500.0,0.179963,0.042623


In [31]:
#def kfold_lightgbm(df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
train_df = df[df['TARGET'].notnull()]
#test_df = df[df['TARGET'].isnull()]


In [32]:
print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
gc.collect()

Starting LightGBM. Train shape: (307507, 261), test shape: (48744, 121)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17036\2390707251.py", line 2, in <module>
    gc.collect()
NameError: name 'gc' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return Form

In [33]:
from lightgbm import LGBMClassifier

nthread=4,
n_estimators=10000,
learning_rate=0.02,
num_leaves=34,
colsample_bytree=0.9497036,
subsample=0.8715623,
max_depth=8,
reg_alpha=0.041545473,
reg_lambda=0.0735294,
min_split_gain=0.0222415,
min_child_weight=39.3259775,
silent=-1,
verbose=-1,

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train,X_test,y_train,y_test = train_test_split(train_df.drop('TARGET', axis =1), train_df['TARGET'])

In [36]:
import mlflow
from sklearn.model_selection import GridSearchCV

In [37]:
#erreur avec ponctuation lgb ne pouvait pas focntionner 
X_train_test.columns = X_train_test.columns.str.replace('[#,@,&,:,\,]', '')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17036\1060894536.py", line 2, in <module>
    X_train_test.columns = X_train_test.columns.str.replace('[#,@,&,:,\,]', '')
NameError: name 'X_train_test' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPy

In [53]:
X_train_test

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17036\2364843524.py", line 1, in <module>
    X_train_test
NameError: name 'X_train_test' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    r

Gridsearch + mlflow

In [38]:
#definition gridsearch
grid ={"n_estimators":[100,1000], "max_depth":[10,15]}#{"n_estimators":np.arange(100,500,150), "max_depth":np.arange(2,10,4)}# 
logreg=LGBMClassifier(zero_as_missing =True)
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train_test,y_train)

for i in range(len(logreg_cv.cv_results_['params'])): # pour chaque parametre essayer
    run_index = i
    cv_results = logreg_cv.cv_results_
    with mlflow.start_run(run_name=str(run_index)) as run:  
            mlflow.log_param("folds", logreg_cv.cv)
            print("Logging parameters")
            params = list(logreg_cv.param_grid.keys())
            for param in params:
                mlflow.log_param(param, cv_results["param_%s" % param][run_index])
            print("Logging metrics")
            for score_name in [score for score in cv_results if "mean_test" in score]:
                mlflow.log_metric(score_name, cv_results[score_name][run_index])
                mlflow.log_metric(score_name.replace("mean","std"), cv_results[score_name.replace("mean","std")][run_index])
            #print("Logging model")        
            #mlflow.sklearn.log_model(logreg_cv.best_estimator_, 'logistic_regression',cv_results[score_name.replace("mean","std")][run_index])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17036\2327209219.py", line 5, in <module>
    logreg_cv.fit(X_train_test,y_train)
NameError: name 'X_train_test' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\ultratb.py", line 1118, in str

test github


test commit 2 github 

exporting model weights

In [39]:
logreg_cv.estimator.fit(X_train,y_train)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17036\4105427791.py", line 1, in <module>
    logreg_cv.estimator.fit(X_train,y_train)
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\lightgbm\sklearn.py", line 967, in fit
    super().fit(X, _y, sample_weight=sample_weight, init_score=init_score, eval_set=valid_sets,
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\lightgbm\sklearn.py", line 748, in fit
    self._Booster = train(
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de

In [40]:
logreg_cv.estimator.predict([X_test.iloc[0,:]])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17036\3897174844.py", line 1, in <module>
    logreg_cv.estimator.predict([X_test.iloc[0,:]])
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\lightgbm\sklearn.py", line 984, in predict
    result = self.predict_proba(X, raw_score, start_iteration, num_iteration,
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\lightgbm\sklearn.py", line 997, in predict_proba
    result = super().predict(X, raw_score, start_iteration, num_iteration, pred_leaf, pred_contrib, **kwargs)
  File "c:\Users

In [41]:
import pickle
pickle_out = open("classifier_2.pkl","wb")
pickle.dump(logreg_cv.estimator,pickle_out)
pickle_out.close()

Modifier le nom des colonnes remplacer espace par _

In [42]:
import re

In [43]:
for dataset in [X_train,X_test]:
     dataset.columns = [re.sub(' ','_',string=col) if ' ' in col else col  for col in dataset.columns]
     dataset.columns = [re.sub(r"[^a-zA-Z0-9]+", '_',col) for col in dataset.columns]

Vérification

In [44]:
[col  for col in X_test.columns if ' ' in col]

[]

In [45]:
for col in (X_train.columns):
    #print(col)
    if X_train[col].dtype == 'int64':
        print(col+ ":"+ 'int')
    if X_train[col].dtype == 'float64':
        print(col+ ":"+ 'float')
    if X_train[col].dtype == 'uint8':
        print(col+ ":"+ 'int')

index:int
SK_ID_CURR:int
CODE_GENDER:int
FLAG_OWN_CAR:int
FLAG_OWN_REALTY:int
CNT_CHILDREN:int
AMT_INCOME_TOTAL:float
AMT_CREDIT:float
AMT_ANNUITY:float
AMT_GOODS_PRICE:float
REGION_POPULATION_RELATIVE:float
DAYS_BIRTH:int
DAYS_EMPLOYED:float
DAYS_REGISTRATION:float
DAYS_ID_PUBLISH:int
OWN_CAR_AGE:float
FLAG_MOBIL:int
FLAG_EMP_PHONE:int
FLAG_WORK_PHONE:int
FLAG_CONT_MOBILE:int
FLAG_PHONE:int
FLAG_EMAIL:int
CNT_FAM_MEMBERS:float
REGION_RATING_CLIENT:int
REGION_RATING_CLIENT_W_CITY:int
HOUR_APPR_PROCESS_START:int
REG_REGION_NOT_LIVE_REGION:int
REG_REGION_NOT_WORK_REGION:int
LIVE_REGION_NOT_WORK_REGION:int
REG_CITY_NOT_LIVE_CITY:int
REG_CITY_NOT_WORK_CITY:int
LIVE_CITY_NOT_WORK_CITY:int
EXT_SOURCE_1:float
EXT_SOURCE_2:float
EXT_SOURCE_3:float
APARTMENTS_AVG:float
BASEMENTAREA_AVG:float
YEARS_BEGINEXPLUATATION_AVG:float
YEARS_BUILD_AVG:float
COMMONAREA_AVG:float
ELEVATORS_AVG:float
ENTRANCES_AVG:float
FLOORSMAX_AVG:float
FLOORSMIN_AVG:float
LANDAREA_AVG:float
LIVINGAPARTMENTS_AVG:float
LIV

In [46]:
from pydantic import BaseModel
from Credit_demands import Credit_demand_columns
# 2. Class which describes Bank Notes measurements
#class Credid_demand_columns(BaseModel):
#    index:int
#    SK_ID_CURR:int

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17036\935303842.py", line 2, in <module>
    from Credit_demands import Credit_demand_columns
ModuleNotFoundError: No module named 'Credit_demands'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\ultratb.py

In [47]:
keys = Credit_demand_columns.__dict__['__fields__'].keys()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17036\56627874.py", line 1, in <module>
    keys = Credit_demand_columns.__dict__['__fields__'].keys()
NameError: name 'Credit_demand_columns' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\

In [48]:
X_test.iloc[0,:]

index                   43153.000000
SK_ID_CURR             149953.000000
CODE_GENDER                 0.000000
FLAG_OWN_CAR                0.000000
FLAG_OWN_REALTY             0.000000
                           ...      
DAYS_EMPLOYED_PERC          0.022425
INCOME_CREDIT_PERC          0.447217
INCOME_PER_PERSON      175500.000000
ANNUITY_INCOME_PERC         0.085026
PAYMENT_RATE                0.038025
Name: 43153, Length: 260, dtype: float64

data_client = {}
data_client.dict(dict[name]==val for name,val in zip(keys,info_client.values))

In [ ]:
info_client =  X_train.iloc[0,:]
data_client = {}
for name,val in zip(keys,info_client.values):
    print(name,val)
    data_client[name]=val 

index 179903.0
SK_ID_CURR 308482.0
CODE_GENDER 0.0
FLAG_OWN_CAR 1.0
FLAG_OWN_REALTY 0.0
CNT_CHILDREN 1.0
AMT_INCOME_TOTAL 67500.0
AMT_CREDIT 127350.0
AMT_ANNUITY 8266.5
AMT_GOODS_PRICE 112500.0
REGION_POPULATION_RELATIVE 0.01885
DAYS_BIRTH -9374.0
DAYS_EMPLOYED -283.0
DAYS_REGISTRATION -4038.0
DAYS_ID_PUBLISH -2022.0
OWN_CAR_AGE 12.0
FLAG_MOBIL 1.0
FLAG_EMP_PHONE 1.0
FLAG_WORK_PHONE 1.0
FLAG_CONT_MOBILE 1.0
FLAG_PHONE 1.0
FLAG_EMAIL 0.0
CNT_FAM_MEMBERS 3.0
REGION_RATING_CLIENT 2.0
REGION_RATING_CLIENT_W_CITY 2.0
HOUR_APPR_PROCESS_START 14.0
REG_REGION_NOT_LIVE_REGION 0.0
REG_REGION_NOT_WORK_REGION 0.0
LIVE_REGION_NOT_WORK_REGION 0.0
REG_CITY_NOT_LIVE_CITY 0.0
REG_CITY_NOT_WORK_CITY 0.0
LIVE_CITY_NOT_WORK_CITY 0.0
EXT_SOURCE_1 nan
EXT_SOURCE_2 0.5761650998327944
EXT_SOURCE_3 0.7713615919194317
APARTMENTS_AVG nan
BASEMENTAREA_AVG nan
YEARS_BEGINEXPLUATATION_AVG nan
YEARS_BUILD_AVG nan
COMMONAREA_AVG nan
ELEVATORS_AVG nan
ENTRANCES_AVG nan
FLOORSMAX_AVG nan
FLOORSMIN_AVG nan
LANDAREA_AVG 

In [49]:
pickle_in = open("classifier_2.pkl","rb") #notre modele importer
classifier_3=pickle.load(pickle_in) 
classifier_3.predict(X_test.iloc[0,:])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17036\4271867952.py", line 3, in <module>
    classifier_3.predict(X_test.iloc[0,:])
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\lightgbm\sklearn.py", line 984, in predict
    result = self.predict_proba(X, raw_score, start_iteration, num_iteration,
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\lightgbm\sklearn.py", line 997, in predict_proba
    result = super().predict(X, raw_score, start_iteration, num_iteration, pred_leaf, pred_contrib, **kwargs)
  File "c:\Users\utilisat

In [50]:
classifier_3

LGBMClassifier(zero_as_missing=True)

In [51]:
classifier_3.feature_importances_

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17036\2277281599.py", line 1, in <module>
    classifier_3.feature_importances_
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\lightgbm\sklearn.py", line 874, in feature_importances_
    raise LGBMNotFittedError('No feature_importances found. Need to call fit beforehand.')
sklearn.exceptions.NotFittedError: No feature_importances found. Need to call fit beforehand.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele

Export X_test pour faire test avec modele

In [ ]:
X_test.to_csv('X_test.csv', index=False)

In [ ]:
t = pd.read_csv('X_test.csv')
t.head()

,index,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,285569,430740,1,0,1,0,180000.0,1078200.0,31653.0,900000.0,...,0,0,1,0,0,0.045683,0.166945,90000.0,0.175850,0.029357
1,46575,153955,1,1,0,0,225000.0,1032133.5,34240.5,891000.0,...,0,0,1,0,0,0.041098,0.217995,225000.0,0.152180,0.033174
2,88768,203063,1,0,1,0,72000.0,180000.0,14220.0,180000.0,...,0,0,1,0,0,0.072463,0.400000,36000.0,0.197500,0.079000
3,228766,364976,1,0,0,1,270000.0,450000.0,22500.0,450000.0,...,0,1,0,0,1,0.069528,0.600000,90000.0,0.083333,0.050000
4,270626,413687,1,1,1,0,270000.0,239850.0,22324.5,225000.0,...,0,1,1,0,0,0.011312,1.125704,270000.0,0.082683,0.093077


In [ ]:
t[t['SK_ID_CURR']==430740]

,index,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,285569,430740,1,0,1,0,180000.0,1078200.0,31653.0,900000.0,...,0,0,1,0,0,0.045683,0.166945,90000.0,0.17585,0.029357


In [ ]:
[classifier_3.predict(t[t['SK_ID_CURR']==430740])[0]]

[0.0]

In [ ]:
import requests

api pour recueillir les prédictions et les feature importance

In [ ]:
url = url = "http://127.0.0.1:8000/predict/"
res = requests.post(url+'430740')

In [ ]:
import json
test_data_retrieved = json.loads(res._content.decode('utf-8'))

In [ ]:
test_data_retrieved['prediction']

[0.0]

In [ ]:
test_data_retrieved['feature_importance']

[51,
 0,
 31,
 6,
 4,
 4,
 42,
 78,
 72,
 77,
 34,
 119,
 106,
 67,
 89,
 58,
 0,
 0,
 12,
 0,
 4,
 0,
 4,
 4,
 17,
 14,
 0,
 0,
 3,
 10,
 1,
 0,
 233,
 176,
 209,
 17,
 19,
 15,
 19,
 13,
 6,
 9,
 11,
 9,
 28,
 9,
 17,
 12,
 10,
 12,
 27,
 18,
 14,
 7,
 1,
 7,
 4,
 2,
 16,
 6,
 10,
 9,
 10,
 9,
 12,
 9,
 3,
 9,
 1,
 3,
 1,
 0,
 12,
 6,
 9,
 6,
 14,
 26,
 13,
 14,
 9,
 19,
 94,
 0,
 19,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 4,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 3,
 4,
 25,
 26,
 26,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 8,
 0,
 0,
 7,
 0,
 1,
 23,
 0,
 4,
 12,
 0,
 2,
 20,
 4,
 2,
 0,
 2,
 0,
 2,
 1,
 6,
 2,
 2,
 1,
 0,
 4,
 1,
 3,
 10,
 4,
 0,
 5,
 0,
 5,
 0,
 2,
 2,
 0,
 0,
 4,
 0,
 0,
 1,
 4,
 4,
 2,
 1,
 2,
 0,
 3,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 4,
 0,
 3,
 0,
 3,
 6,
 0,
 1,
 2,
 0,
 6,
 0,
 1,
 6,
 0,
 0,
 9,
 0,
 1,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,

Afficher les prédictions